# Instalações e importações

In [1]:
%pip install boto3

  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 870.0 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 17.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.5 MB/s eta 0:00:00
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 535.4 kB/s eta 0:00:00a 0:00:01
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.3 MB/s eta 0:00:0000:0100:01
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import boto3
from io import StringIO
import re
import json
import datetime

In [7]:
from dic_pof import *

In [8]:
aluguel_estimado_dic

{'UF': 'estado',
 'ESTRATO_POF': 'estrato_amostral',
 'TIPO_SITUACAO_REG': 'tipo_domicilio',
 'COD_UPA': 'codigo_amostragem',
 'NUM_DOM': 'numero_domicilio',
 'NUM_UC': 'numeroconsumo',
 'QUADRO': 'codigo_despesa',
 'V9001': 'codigo_despesa_fixo',
 'V9002': 'forma_aquisicao',
 'V8000': 'valor_aluguel',
 'V9010': 'mes_despesa',
 'V9011': 'duracao_despesa',
 'DEFLATOR': 'fator_deflacionario',
 'V8000_DEFLA': 'aluguel_deflacionado',
 'COD_IMPUT_VALOR': 'imputacao_valor',
 'FaTOR_ANUALIZACAO': 'fator_anualizacao',
 'PESO': 'fator_expansao',
 'PESO_FINAL': 'peso_ajustado',
 'RENDA_TOTAL': 'renda_total_uc'}

# Tratamento e load dos dados

- IMPORTANTE: tenha em mente que as seguintes funções requerem um arquivo em CSV, delimitado por vírgula e não com ponto e vírgula. Caso seu arquivo esteja com ponto e vírgula, altere o parâmetro separação ','.

In [23]:
class Tl:
    def __init__(self, _df):
        self.df = pd.read_csv(_df, delimiter=',')
        self.ja_foi_convertido = False
        self.ja_enviado = False

    def informacao_csv_atual(self):
        return self.df

    def tratar_nulos(self):
        self.df.fillna(0, inplace=True)


    def converter_arquivo(self, type):
        assert self.ja_foi_convertido == False, "Arquivo ja foi convertido."

        json_str = self.df.apply(lambda x: json.dumps(x.to_dict()), axis=1)

        df_json = pd.DataFrame({'value': json_str, 'type': type})

        self.df = df_json
        self.ja_foi_convertido = True

    def substituicao_nomenclaturas(self, dicionario_de_substituicao):
        self.df.rename(columns=dicionario_de_substituicao, inplace=True)


    def load_s3(self, bucket_name, object_name, _aws_access_key_id , _aws_secret_access_key, token):

        if self.ja_enviado == True:
          reposta = input("arquivo ja enviado, deseja enviar mais uma vez?,S/N:")
          assert reposta == "S", "Arquivo não foi mandado para a S3"

        s3_client = boto3.client('s3', aws_access_key_id= _aws_access_key_id, aws_secret_access_key= _aws_secret_access_key, aws_session_token=token)
        csv_buffer = StringIO()

        assert self.ja_foi_convertido == True, "Arquivo precisa estar convertido, rodar a funcao converter_arquivo() para converter os dados"

        self.df['ingestion_date'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


        self.df.to_csv(csv_buffer, index=False)


        s3_client.put_object(Bucket=bucket_name, Key=object_name, Body=csv_buffer.getvalue())

        self.ja_enviado = True


## Demonstrativo, com csv modificado

In [24]:

inep = Tl("condicao_de_vida.csv")

In [25]:
inep.informacao_csv_atual()

,UF,ESTRATO_POF,TIPO_SITUACAO_REG,COD_UPA,NUM_DOM,NUM_UC,COD_INFORMANTE,V6101,V6102,V6103,...,V6115,V6116,V6117,V6118,V6119,V6120,V6121,PESO,PESO_FINAL,RENDA_TOTAL
0,11,1103,1,110005400,1,1,1,5,4000,1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.806669,372.984516,11254.75
1,11,1103,1,110005400,2,1,1,3,3500,1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.806669,372.984516,10828.07
2,11,1103,1,110005400,4,1,1,5,5000,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.806669,372.984516,4769.13
3,11,1103,1,110005400,5,1,1,3,1800,800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.806669,372.984516,2313.61
4,11,1103,1,110005400,6,1,1,5,6000,1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.806669,372.984516,6596.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58034,53,5306,1,530025746,8,1,1,1,3800,500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.403820,620.212733,2692.17
58035,53,5306,1,530025746,9,1,1,3,15000,3000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.403820,620.212733,33566.71
58036,53,5306,1,530025746,10,1,2,5,5000,850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.403820,620.212733,39756.66
58037,53,5306,1,530025746,12,1,1,4,12000,3000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.403820,620.212733,44369.03


In [26]:
inep.substituicao_nomenclaturas(condicoes_vida_dic)

In [27]:
inep.converter_arquivo("pof_condicoes_vida")

In [28]:
inep.informacao_csv_atual()

,value,type
0,"{""estado"": 11.0, ""estrato_amostral"": 1103.0, ""...",pof_condicoes_vida
1,"{""estado"": 11.0, ""estrato_amostral"": 1103.0, ""...",pof_condicoes_vida
2,"{""estado"": 11.0, ""estrato_amostral"": 1103.0, ""...",pof_condicoes_vida
3,"{""estado"": 11.0, ""estrato_amostral"": 1103.0, ""...",pof_condicoes_vida
4,"{""estado"": 11.0, ""estrato_amostral"": 1103.0, ""...",pof_condicoes_vida
...,...,...
58034,"{""estado"": 53.0, ""estrato_amostral"": 5306.0, ""...",pof_condicoes_vida
58035,"{""estado"": 53.0, ""estrato_amostral"": 5306.0, ""...",pof_condicoes_vida
58036,"{""estado"": 53.0, ""estrato_amostral"": 5306.0, ""...",pof_condicoes_vida
58037,"{""estado"": 53.0, ""estrato_amostral"": 5306.0, ""...",pof_condicoes_vida


In [31]:
inep.load_s3("grupo4-bigd-pof", "condicao_de_vida","_aws_access_key_id", "_aws_secret_access_key", "token")